<a href="https://colab.research.google.com/github/samherreid/DebrisCoverMetrics/blob/main/See_G.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## See-G(IS code)

By Sam Herreid 

October 2022

See-G is a tool for visual learners to see arcpy functions plotted in space and turned into a sequential video to troubleshoot, vision board, identify inefficiencies and communicate many-step methods quickly and exhaustively in talks and paper supplemental information.

In order to use See-G, you have to set up an ArcGIS Pro project with a layout following https://pro.arcgis.com/en/pro-app/latest/arcpy/mapping/tutorial-getting-started-with-arcpy-mp.htm and add an empty text box somewhere in the layout. The text box will be repopulated each run with the current function, line number and file name.

The last block is an example of how See-G can be used in your code.

In [ ]:
baseMap = r"" # .shp (optional)
arcGisProject = r"" # .aprx
frameFolderPath = r""

In [ ]:
imageCount = 0 # inititalize (or reset) the count

In [ ]:
import arcpy
import sys
from inspect import currentframe

In [ ]:
baseMapLayer = arcpy.MakeFeatureLayer_management(baseMap, "base_lyr").getOutput(0)

In [ ]:
def imgCount():
  imageCount = globals()['imageCount'] + 1
  globals()['imageCount'] = imageCount
  return str(globals()['imageCount']).zfill(4)

In [ ]:
def lineNumber():
  return currentframe().f_back.f_lineno

In [ ]:
def makeFrame(featureToAdd, useSeeG, arcGisProject, baseMapLayer, frameFolderPath, lineNumber, functionName):
  if useSeeG == 'yes':
    aprx = arcpy.mp.ArcGISProject(arcGisProject)
    m = aprx.listMaps("Map")[0]
    feat_lyr = arcpy.MakeFeatureLayer_management(featureToAdd, "feat_lyr").getOutput(0)
    map_lyr = m.addLayer(baseMapLayer)[0]
    map_lyr = m.addLayer(feat_lyr, "TOP")[0]
    lyr = m.listLayers(feat_lyr)[0]
    lyr_base = m.listLayers(baseMapLayer)[0]

    # set base map layer color
    sym = lyr_base.symbology
    colorRamp = aprx.listColorRamps("Cyan to Purple")[0]
    sym.renderer.colorRamp = colorRamp
    lyr_base.symbology = sym

    lyt = aprx.listLayouts()[0]
    mf = lyt.listElements("mapframe_element")[0]
    mf.camera.setExtent(mf.getLayerExtent(lyr, False, True))

    # add line number from code by cloning an existing test element in .aprx
    elm = lyt.listElements('TEXT_ELEMENT')[0]
    elm.text = functionName+": Line "+str(lineNumber)+" - "+featureToAdd.split("\\")[-1]

    lyt.exportToPNG(frameFolderPath+"\\frame"+imgCount()+".png")
    arcpy.Delete_management("feat_lyr")
    del feat_lyr, map_lyr, lyr, lyt, mf, elm, aprx

With the above setup and functons imported, add the makeFrame line of code to every instance in your code where you want a methods map frame saved.  

In [ ]:
useSeeG = 'yes' # put this commant at the head of your code and pass it as a variable in any of your functions. Setting useSeeg to [], 'no', or anything else will run your code without making See-G frames

arcpy.someFunction(input,output) # I think only .shp works right now, but grids should be easy to accomodate

# only output should change, the other variables will be the same unless you want to change the base map layer
makeFrame(output, useSeeG, arcGisProject, baseMapLayer, frameFolderPath, lineNumber(), sys._getframe().f_code.co_name) 